In [5]:
import numpy as np
import pandas as pd
import time
from qiskit import Aer, execute
from qiskit.circuit.library import GroverOperator
from qiskit.algorithms import AmplificationProblem, Grover
from qiskit.utils import QuantumInstance

ImportError: cannot import name 'Aer' from 'qiskit' (C:\Users\66023\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\qiskit\__init__.py)

In [ ]:
# Load the CSV file
data = pd.read_csv('dataset_small.csv')

# Define the target condition (this is what you're searching for)
# For example, let's say we are searching for rows where the 'Product_Type' is 'Soft Drink'
def target_condition(row):
    return row['Product_Type'] == 'Soft Drink'

# Classical search (for comparison)
def classical_search(data):
    for index, row in data.iterrows():
        if target_condition(row):
            return index, row
    return None, None

In [ ]:
# Grover's algorithm using Qiskit
def grover_search_qiskit(data):
    num_items = len(data)
    
    # Define the oracle: A classical function that checks if an item matches the target condition
    def oracle(index):
        return target_condition(data.iloc[index])
    
    # Set up the quantum circuit
    # The number of qubits should be able to represent all indices
    num_qubits = int(np.ceil(np.log2(num_items)))
    
    # Initialize Qiskit's Grover instance
    oracle_bitstring = [oracle(i) for i in range(num_items)]
    oracle_bitstring = [int(bit) for bit in oracle_bitstring]
    
    # Create the Oracle for Grover's algorithm
    problem = AmplificationProblem(oracle=oracle_bitstring, is_good_state=oracle_bitstring)
    
    # Use Aer (Qiskit's simulator) as the backend
    backend = Aer.get_backend('qasm_simulator')
    
    grover = Grover(quantum_instance=QuantumInstance(backend))
    
    # Run Grover's algorithm
    result = grover.amplify(problem)
    
    # Get the most likely index
    result_index = int(result.top_measurement, 2)
    return result_index, data.iloc[result_index]

In [ ]:
# Measure and run the classical search
start_time = time.time()
classical_result_index, classical_result_row = classical_search(data)
end_time = time.time()
classical_time = end_time - start_time
print(f"Classical Search Result: Index {classical_result_index}, Row: {classical_result_row}")
print(f"Classical Search Processing Time: {classical_time:.6f} seconds")

print('\n\n')

# Measure and run the Grover's algorithm with Qiskit
start_time = time.time()
grover_result_index, grover_result_row = grover_search_qiskit(data)
end_time = time.time()
grover_time = end_time - start_time
print(f"Grover Search Result: Index {grover_result_index}, Row: {grover_result_row}")
print(f"Grover Search Processing Time: {grover_time:.6f} seconds")